In [1]:
import pandas as pd
import numpy as np

In [2]:
# u.user 파일을 DataFrame으로 읽기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [3]:
# u.item 파일을 DataFrame으로 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
# u.data 파일을 DataFrame으로 읽기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


# 인기제품 방식

In [5]:
# Best-seller 추천 
def recom_movie1(n_items):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    # 각 영화의 평점평균인 movie_mean을 내림차순으로 정렬하고, 그 중 상위부터 지정된 계수(n_item) 만큰을 movie_sort에 저장 
    # 여기서 mean과 sort는 pandas series 형식이라, 값과 index가 같이 저장된다
    recom_movies = movies.loc[movie_sort.index]
    # movies 에 있는 영화 중에 선정된 5개의 영화에 대한 정보 저장
    recommendations = recom_movies['title']
    # 선정된 5개의 영화에서 영화 제목을 골라서 반환, recom_movies은 Dataframe 형식으로 제목만 ["title"] 뽑을 수 잇다 
    return recommendations

movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
# ratingsdp 에 있는 평가 데이터를 pandas dataframe의 groupby() 메소드를 활용해서 각 영화(movie_id)별 평점(rating)의 평균(mean)을 
# 구한 후에 저장, 이때  movie_meandms 은 pandas의 series 형식을 갖는다 
recom_movie1(5)
# 함수를 호출해서 인자(parameter)인 n_items을 5로 설정, 이 값은 추천 받을 아이템 숫자이다 

movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
Name: title, dtype: object

In [6]:
#  동일한 함수를 한줄로 정의 
def recom_movie2(n_items):
   return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

recom_movie2(5)

movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
Name: title, dtype: object

# 정확도 계산 

In [7]:
# RMSE(Root Mean Squared Error)
# 각 예측값과 해당 실제값의 차이(error)를 제곱(squared)의 평균(maen)의 제곱근(root)
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [8]:
rmse = [] # 결과를 저장할 RMSE 변수를 초기화 
for user in set(ratings.index):
    y_true = ratings.loc[user]['rating']  # 각 사용자가 평가한 모든 영화의 평점을 y_ture에 저장
    y_pred = movie_mean[ratings.loc[user]['movie_id']] 
    # 해당 사용자가 평가한 영화의 평점 평균을 y_pred에 저장, 
    # 현재 사용하는 추천알고리즘이  bset-seller 방식이므로 평점 평균이 해당 영화의 예측값이라고 할 수 있다 
    accuracy = RMSE(y_true, y_pred)
    # 실제값과 에측값으로 RMSE를 rmse리스트에 추가 
    rmse.append(accuracy)
print(np.mean(rmse))


0.996007224010567
